# Exploratory data analysis

In [1]:
import pandas as pd
import numpy as np

## Import Movie Lens Data Base

In [2]:
#importar databasese Movie Lens e checar tamanhos
data_path = '../raw_data/ml-25m/'
g_scores = pd.read_csv(data_path + 'genome-scores.csv')#, nrows = 1000000)
g_tags = pd.read_csv(data_path +'genome-tags.csv')#, nrows = 1000000)
links = pd.read_csv(data_path + 'links.csv')
movies = pd.read_csv(data_path + 'movies.csv')
ratings = pd.read_csv(data_path + 'ratings.csv') #nrows = 1000000)
tags = pd.read_csv(data_path + 'tags.csv')

In [3]:
print(f'Formato da base g_scores:',(g_scores.shape))
print(f'Formato da base g_tags:',(g_tags.shape))
print(f'Formato da base links:',(links.shape))
print(f'Formato da base movies:',(movies.shape))
print(f'Formato da base ratings:',(ratings.shape))
print(f'Formato da base tags:',(tags.shape))

Formato da base g_scores: (15584448, 3)
Formato da base g_tags: (1128, 2)
Formato da base links: (62423, 3)
Formato da base movies: (62423, 3)
Formato da base ratings: (25000095, 4)
Formato da base tags: (1093360, 4)


## Análise exploratória Movie Lens Data Base

### Genome-scores:


In [4]:
g_scores.head(4)

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575


In [5]:
print(g_scores.isnull().sum())
g_scores.info()

movieId      0
tagId        0
relevance    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15584448 entries, 0 to 15584447
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 356.7 MB


### Genome-tags:

In [6]:
g_tags.head(4)

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s


In [7]:
print(g_tags.isnull().sum())
g_tags.info()

tagId    0
tag      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


### Links:

In [8]:
links.head(4)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0


In [9]:
print(links.isnull().sum())
links.info()

movieId      0
imdbId       0
tmdbId     107
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  62423 non-null  int64  
 1   imdbId   62423 non-null  int64  
 2   tmdbId   62316 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.4 MB


### Movies:

In [10]:
movies.head(4)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance


In [11]:
print(movies.isnull().sum())
movies.info()

movieId    0
title      0
genres     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


### Ratings:

In [12]:
ratings.head(4)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820


In [13]:
print(ratings.isnull().sum())
ratings.info()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


### Tags:


In [14]:
tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [15]:
print(tags.isnull().sum())
tags.info()

userId        0
movieId       0
tag          16
timestamp     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093360 entries, 0 to 1093359
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1093360 non-null  int64 
 1   movieId    1093360 non-null  int64 
 2   tag        1093344 non-null  object
 3   timestamp  1093360 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.4+ MB


We can exclude 16 tags because we have 1093360 tags on the movies

Which movies don't have tags?

### Movies without TMDB ID

In [16]:
#criação de dataframe missing_movies
missing_movies = links.copy()
#criação de campo 'empty' no dataframe
missing_movies['empty'] = missing_movies.tmdbId.notnull()
#check a na quantidade de valores sem tmdbId
print(missing_movies['empty'].value_counts())
missing_movies = missing_movies[missing_movies['empty'] == False]
missing_movies = missing_movies.merge(movies,on='movieId')
missing_movies.head(5)

True     62316
False      107
Name: empty, dtype: int64


,movieId,imdbId,tmdbId,empty,title,genres
0,721,114103,NaN,False,Halfmoon (Paul Bowles - Halbmond) (1995),Drama
1,730,125877,NaN,False,Low Life (1994),Drama
2,770,38426,NaN,False,Costa Brava (1946),Drama
3,791,113610,NaN,False,"Last Klezmer: Leopold Kozlowski, His Life and ...",Documentary
4,1107,102336,NaN,False,Loser (1991),Comedy


We can exclude the 107 movies that don't have tmdbId!

### Tags without movies

In [17]:
movie_ids_without_tags = tags[tags.tag.isnull()].movieId.count()

In [18]:
print(f'Tags without movies: {movie_ids_without_tags}')

Tags without movies: 16


We can exclude 16 entries from Tags as they are non significant

### Standardizing movie genres:


In [19]:
movies = movies.copy()
movies['genres'] = movies['genres'].str.replace("|"," ")

In [20]:
#função para transformar string em lista 
def movie_genres(x):
    # remover (no genres listed) 
    if x == '(no genres listed)':
        return ""
    else:
        return x.split()   

In [21]:
movies['genres'] = movies['genres'].apply(movie_genres)
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"


In [22]:
movies.head(4)

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"


In [23]:
test = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
test = [genre.lower() for genre in test]
test

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'film-noir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'romance',
 'sci-fi',
 'thriller',
 'war',
 'western']

In [24]:
from sklearn.preprocessing import MultiLabelBinarizer

labels = movies['genres']
mlb = MultiLabelBinarizer()
results = mlb.fit_transform(labels)
print(mlb.classes_)
col = list(mlb.classes_)
col = [genre.lower() for genre in col]
print(col)
print(results)

movies = movies.join(pd.DataFrame(results, 
                     columns=col,
                    index = movies.index))

['Action' 'Adventure' 'Animation' 'Children' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'IMAX' 'Musical'
 'Mystery' 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western']
['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western']
[[0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]]


In [25]:
movies.head(4)

,movieId,title,genres,action,adventure,animation,children,comedy,crime,documentary,...,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


## Movie Lens - Data cleaning

### Merging Movies and Link database and removing 107 movies without TMDB ID

In [26]:
movies = links.merge(movies, on='movieId', how='right')
print(movies.shape)
movies.head(1)

(62423, 24)


,movieId,imdbId,tmdbId,title,genres,action,adventure,animation,children,comedy,...,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
0,1,114709,862.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
movies = movies.dropna(axis=0, subset=['tmdbId'])

In [28]:
print(links.movieId.max())
print(links.imdbId.max())
print(links.tmdbId.max())
print("int16 max: 32.767")
print("int32 max: 2.147.483.647")

209171
11170942
646282.0
int16 max: 32.767
int32 max: 2.147.483.647


In [29]:
movies.movieId = movies.movieId.astype(np.int32)
movies.imdbId = movies.imdbId.astype(np.int32)
movies.tmdbId = movies.tmdbId.astype(np.int32)

In [30]:
movies.head(4)

,movieId,imdbId,tmdbId,title,genres,action,adventure,animation,children,comedy,...,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
0,1,114709,862,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,113497,8844,Jumanji (1995),"[Adventure, Children, Fantasy]",0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,113228,15602,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,114885,31357,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


### Removing NaN entries from tags

In [31]:
tags = tags.dropna(axis=0, subset=['tag'])
tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1093344 entries, 0 to 1093359
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1093344 non-null  int64 
 1   movieId    1093344 non-null  int64 
 2   tag        1093344 non-null  object
 3   timestamp  1093344 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 41.7+ MB


## Data from API TMDB

### Import data from API TMDB

In [32]:
movies_details = pd.read_csv("../raw_data/tmdb-api/movies_details.csv")
movies_details = movies_details.drop(columns=['Unnamed: 0'])
movies_details.head(4)

print(f'Number of entries that the API brought us: {movies_details.shape[0]}')

Number of entries that the API brought us: 62316


### Remove entries without detail

In [33]:
movies_without_details = movies_details[movies_details.movie_details == '(None, None, None, None, None, None, None)'].copy()

print(f'Number of entries without details: {movies_without_details.shape[0]}')

Number of entries without details: 620


In [34]:
movies_without_details = movies_without_details.merge(movies, on=['movieId'])
movies_without_details[['title', 'tmdbId_x']].to_html("movies_without_details.html")

After analysing the movies that didn't had details we discovered that they were in fact TV Series, an thent we should have look for their details on another API. So we decidid to exclude theses TV series from our dataset.

In [35]:
movies_details = movies_details[movies_details.movie_details != '(None, None, None, None, None, None, None)'].copy()

print(f'Number of entries with details: {movies_details.shape[0]}')

Number of entries with details: 61696


In [36]:
movies_details.reset_index(drop=True, inplace=True)

### Create columns for every key received from the API

In [37]:
import ast

detail = movies_details.iloc[0].movie_details
columns = list(ast.literal_eval(detail).keys())
columns

['language',
 'overview',
 'popularity',
 'release',
 'runtime',
 'vote_average',
 'vote_count']

In [38]:
def convert_to_dict(details):
    dict_details = ast.literal_eval(details)
    return dict_details

In [39]:
movies_details.movie_details = movies_details.movie_details.apply(convert_to_dict)

In [40]:
columns_of_details = pd.DataFrame.from_records(movies_details.movie_details.values, columns=columns)

In [41]:
movies_details[columns] = columns_of_details

In [42]:
movies_details_export = movies_details[['movieId', 'language', 'overview', 'popularity', 'release', 'runtime', 'vote_average', 'vote_count']].copy()

In [43]:
movies_details_export

,movieId,language,overview,popularity,release,runtime,vote_average,vote_count
0,1,en,"Led by Woody, Andy's toys live happily in his ...",70.910,1995-10-30,81.0,7.9,13173
1,2,en,When siblings Judy and Peter discover an encha...,11.919,1995-12-15,104.0,7.2,7711
2,3,en,A family wedding reignites the ancient feud be...,13.122,1995-12-22,101.0,6.6,217
3,4,en,"Cheated on, mistreated and stepped on, the wom...",5.123,1995-12-22,127.0,6.2,79
4,5,en,Just when George Banks has recovered from his ...,15.142,1995-12-08,106.0,6.2,446
...,...,...,...,...,...,...,...,...
61691,209157,nl,During a scorching summer in a Belgian-Dutch b...,6.605,2018-07-12,100.0,6.2,26
61692,209159,pt,Nineteen people with differing degrees of visu...,1.131,2001-10-20,73.0,7.1,14
61693,209163,hu,"33-years old Tamás Merthner is heartbroken, af...",0.834,2018-12-27,97.0,6.7,6
61694,209169,en,A Girl Thing is a mini-series that revolves ar...,2.508,2001-01-19,237.0,3.4,4


### Export Api data to CSV

In [44]:
movies_details_export.to_csv('../raw_data/treated-tmdb-api/movies_details.csv', index=False)

# Test with Collaborative filtering model

Collaborative filtering based recommendation engine is a family of algorithms that tries to find similar users based on similar preferences, actions and activities . It then looks at the movies for one user and recommends it to a similar user. Let’s take for example user A who is similar to user B, we know they are both similar because they both like the same video games, comic books, etc., if user A has seen a movie that user B hasn’t, then this recommendation engine will recommend that movie to user B.

In [45]:
movies.head()

,movieId,imdbId,tmdbId,title,genres,action,adventure,animation,children,comedy,...,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
0,1,114709,862,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,113497,8844,Jumanji (1995),"[Adventure, Children, Fantasy]",0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,113228,15602,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,114885,31357,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,113041,11862,Father of the Bride Part II (1995),[Comedy],0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


## Create dataset crossing users, movies and ratings

Check if all the movieIds are unique

In [47]:
len(movies.movieId)

62316

In [48]:
len(movies.movieId.unique())

62316

In [71]:
ratings_per_user = ratings.userId.value_counts()
users_that_rated_at_most_35_movies = ratings_per_user[ratings_per_user<=35].index
ratings_l35 = ratings[ratings.userId.isin(users_that_rated_at_most_35_movies)]
ratings_l35

,userId,movieId,rating,timestamp
1253,6,161,2.0,945141530
1254,6,260,5.0,945142152
1255,6,318,5.0,945142558
1256,6,527,5.0,945142152
1257,6,593,4.0,945142787
...,...,...,...,...
24998414,162531,338,3.0,828993342
24998415,162531,349,5.0,828993342
24998416,162531,431,5.0,828993342
24998417,162531,434,4.0,828993342


In [72]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

movie_c = CategoricalDtype(sorted(ratings_l35.movieId.unique()), ordered=True)
user_c = CategoricalDtype(sorted(ratings_l35.userId.unique()), ordered=True)

row = ratings_l35.userId.astype(user_c).cat.codes
col = ratings_l35.movieId.astype(movie_c).cat.codes

sparse_matrix = csr_matrix((ratings_l35["rating"], (row, col)), \
                           shape=(user_c.categories.size, movie_c.categories.size))

In [73]:
sparse_matrix.shape

(40282, 13899)

In [ ]:
# df = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, index=user_c.categories, columns=movie_c.categories)
# df

In [ ]:
# df.index.nunique()

In [84]:
from sklearn.cluster import MiniBatchKMeans, KMeans

mbkm = MiniBatchKMeans(n_clusters=10000)
mbkm.fit(sparse_matrix)

MiniBatchKMeans(n_clusters=10000)

In [85]:
pd.Series(mbkm.labels_).value_counts()

8346    2481
9147    1795
3471    1174
181      663
2815     632
        ... 
9648       1
7575       1
3477       1
1428       1
0          1
Length: 9803, dtype: int64

In [88]:
new_user = pd.DataFrame([np.zeros(len(movie_c.categories))], columns=movie_c.categories)
# Toy Story (1995)
new_user[1] = 1
# Jumanji (1995)
new_user[2] = 1
# Goldeneye
new_user[10] = 1
# Dark Knight
new_user[58559] = 1
# Lord of the rings
new_user[4993] = 1
# Memento
new_user[4226] = 1
# Kingsmen
new_user[119145] = 1
# Midnight in paris
new_user[86882] = 1
# Mule
new_user[195921] = 1
# 50 Shades
new_user[168712] = 5
# X-Men
new_user[3793] = 5
# Into the wild
new_user[55247] = 1
new_user

,1,2,3,4,5,6,7,8,9,10,...,207025,207027,207077,207117,207189,207309,207311,207501,207888,208006
0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
def make_user(movieId, rating):
    acc = 0
    new_user = pd.DataFrame([np.zeros(13899)], columns=movie_c.categories)
    for id in movieId:
        new_user[id] = rating[acc]
        acc += 1
    return new_user

In [94]:
new_user2 = make_user([7,48,94,534,132660,1721,164909,2291,2558,33679,595,130073,2706,7293,1380,45672,31685,587,4246,84374],
             [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])

In [89]:
mbkm.predict(new_user)

array([8346], dtype=int32)

In [95]:
mbkm.predict(new_user2)

array([8346], dtype=int32)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=10000)

svd.fit(sparse_matrix)
sparse_matrix_svd = svd.transform(sparse_matrix)

In [ ]:
mbkm_svd = MiniBatchKMeans(n_clusters=1000)
mbkm_svd.fit(sparse_matrix_svd)

In [ ]:
pd.Series(mbkm_svd.labels_).value_counts()

In [ ]:
new_user_svd = svd.transform(new_user)

mbkm_svd.predict(new_user_svd)

In [ ]:
new_user2_svd = svd.transform(new_user2)

mbkm_svd.predict(new_user2_svd)

In [ ]:
movies

In [ ]:
ratings.value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.histplot(x=ratings.userId.value_counts())
plt.xlim(0,200)

In [ ]:
sns.histplot(x=ratings.userId.value_counts())
plt.xlim(200,500)

In [ ]:
ratings_per_user = ratings.userId.value_counts()

In [ ]:
sns.histplot(x=ratings_per_user)
plt.xlim(20,50)

In [ ]:
print(f'total number of users: {len(ratings_per_user)}')

In [ ]:
print(f'number of users that rated up to 35 movies: {ratings_per_user[ratings_per_user <=35].shape[0]}')

In [ ]:
users_that_rated_at_most_35_movies = ratings_per_user[ratings_per_user <=35].index
users_that_rated_at_most_35_movies

In [ ]:
list(users_that_rated_at_most_35_movies)

In [ ]:
# ratings_up_to_35 = ratings[ratings.userId in list(users_that_rated_at_most_35_movies)]
# ratings_up_to_35